In [ ]:
import boto3
import getpass
import numpy as np
from preconfig import Preconfig
import pandas as pd
from pathlib import Path
from IPython.display import Image
from simulariumio.cytosim import CytosimConverter, CytosimData, CytosimObjectInfo
from simulariumio import MetaData, DisplayData, DISPLAY_TYPE, ModelMetaData, InputFileData, ScatterPlotData


# 1. Upload config files to S3

In [ ]:
# Preconfig class allows us to parse a template file and generate a list of config files.
# Two loops puts the generated config files for a given number of repeats in S3.
preconfig = Preconfig()
path_to_template = '../templates/vary_compress_rate.cym.tpl'
configs = preconfig.parse(path_to_template,{})
s3_client = boto3.client("s3")
bucket = 'cytosim-working-bucket'
num_repeats = 5
job_names = []
buffered = np.empty((len(configs)), dtype=object)
for index, config in enumerate(configs):
    job_name = config[:-4]
    job_names.append(job_name)
    for repeat in range(num_repeats):
        opened_config = open(config, "rb")
        config_name = f'{job_name}/config/{job_name}_{repeat}.cym' 
        s3_client.put_object(Bucket=bucket, Key=config_name, Body=opened_config)
job_names

# 2a. Specify job definition

In [ ]:
job_definition_arn = "job_definition_arn"

# 2b. Create and register job definition

In [ ]:
# Parameters for a job definition
from container_collection.batch.register_batch_job import register_batch_job
from container_collection.batch.make_batch_job import make_batch_job
job_definition_name = "karthikv_cytosim_varycompressrate"
image = "simularium/cytosim:latest"
vcpus = 1
memory = 7000
bucket_name = "s3://cytosim-working-bucket/"
simulation_name = ''

In [ ]:
account = getpass.getpass()

In [ ]:
# Creating job definitions with make_batch_job
# Submitting job definitions with register_batch_job
jobs = np.empty(len(configs))
job_definitions = np.empty((len(configs)), dtype=object)
for index in range(len(configs)):
    print(index)
    simulation_name = job_names[index]
    print(simulation_name)
    job_definition = make_batch_job(f"cytosim-test-varycompressrate-{str(index)}", 'simularium/cytosim:latest', account, 'us-west-2', 'karthikv', 1, 7000, 's3://cytosim-working-bucket/')
    registered_jd = register_batch_job(job_definition)
    job_definitions[index] = registered_jd
    break

        


# 3. Submit job

In [ ]:
# Submit batch job allows us to submit a batch job with a given job definition and job name.
from container_collection.batch.submit_batch_job import submit_batch_job

In [ ]:
new_configs = configs[:4]
new_configs

In [ ]:
# Parameters for our batch job [size indicates our desired number of repeats]
job_name = "cytosim-varycompressrate"
user = "karthikv"
queue = "general_on_demand"
size = 5

In [ ]:
new_configs
job_name

In [ ]:
# Loop to submit our batch jobs [index * size for total number of simulations]
for index in range(len(new_configs)):
    print(index)
    print(f'{job_name}-completerun-config{index}')
    submit_batch_job(name=f'{job_name}-completerun-config{index}', job_definition_arn=job_definitions[index],user=user,queue=queue,size=size)
    

# 4. Monitor job status

In [ ]:
# TODO: check job status, print progress bar
from container_collection.batch.check_batch_job import check_batch_job


# 5. Load results

In [ ]:
from subcell_analysis.cytosim.post_process_cytosim import create_dataframes_for_repeats
import pandas as pd

In [ ]:
bucket_name = 'cytosim-working-bucket'
num_repeats = 5
configs = ['vary_compress_rate0006']

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
create_dataframes_for_repeats(bucket_name, num_repeats, configs)

In [ ]:
from subcell_analysis.compression_workflow_runner import run_workflow,  plot_metric, plot_metric_list
from subcell_analysis.compression_analysis import (
    COMPRESSIONMETRIC,
)

In [ ]:
import pandas as pd

In [ ]:
#TODO: Run metric calculations on repeats.
num_repeats = 5
outputs = [None] * num_repeats
for repeat in range(num_repeats):
    all_output = pd.read_csv(f'dataframes/actin-forces0_{repeat}.csv')
    outputs[repeat] = run_workflow(all_output, [COMPRESSIONMETRIC.PEAK_ASYMMETRY, COMPRESSIONMETRIC.AVERAGE_PERP_DISTANCE, COMPRESSIONMETRIC.NON_COPLANARITY, COMPRESSIONMETRIC.TOTAL_FIBER_TWIST, COMPRESSIONMETRIC.SUM_BENDING_ENERGY])

In [ ]:
import matplotlib.pyplot as plt
metrics = [COMPRESSIONMETRIC.AVERAGE_PERP_DISTANCE, COMPRESSIONMETRIC.TOTAL_FIBER_TWIST, COMPRESSIONMETRIC.SUM_BENDING_ENERGY, COMPRESSIONMETRIC.PEAK_ASYMMETRY, COMPRESSIONMETRIC.NON_COPLANARITY]
for metric in metrics:
    fig, ax = plt.subplots()
    for repeat in range(num_repeats):
        metric_by_time = outputs[repeat].groupby(["time"])[metric].mean()
        ax.plot(metric_by_time, label=f"repeat {repeat}")
    ax.legend()
    ax.set_xlabel("time")
    ax.set_ylabel(metric.value)
    ax.set_title(f"{metric.value} by time")

## 6. Generate Simularium Outputs

In [ ]:
from subcell_analysis.cytosim.post_process_cytosim import cytosim_to_simularium
s3_client = boto3.client("s3")
s3_client.download_file("cytosim-working-bucket", "vary_compress_rate0006/outputs/2/fiber_segment_curvature.txt", "fiber_segment_curvature.txt")
input_data = cytosim_to_simularium("fiber_segment_curvature.txt")


In [ ]:

cytosim_converter = CytosimConverter(input_data)
repeat = 0
for metric in metrics:
    metric_by_time = outputs[repeat].groupby(["time"])[metric].mean()
    cytosim_converter.add_plot(
    ScatterPlotData(
        title=f"{metric} over time",
        xaxis_title="Time",
        yaxis_title=metric.value,
        xtrace=np.arange(len(metric_by_time))*1E-5,
        ytraces={
            f"repeat {repeat}": metric_by_time,
        },
    )
)

cytosim_converter.save("vary_compress_rate_0006_replicate0")

In [ ]:
# TODO: All repetitions in same simularium visualization???